<a href="https://colab.research.google.com/github/Maruf-16203091/Flowers-Prediction/blob/main/flowers_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [3]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
dataset = ImageFolder(root='/content/drive/MyDrive/ML_Datasets/flowers', transform=transform)


In [6]:
subset_size = 4000
if len(dataset) > subset_size:
    dataset, _ = random_split(dataset, [subset_size, len(dataset)-subset_size])


In [7]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)


In [8]:
classes = dataset.dataset.classes
print(f"Classes: {classes}")

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=len(classes)):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)     # 64x64x3 → 64x64x16
        self.pool = nn.MaxPool2d(2, 2)                 # 64x64 → 32x32
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)    # 32x32x16 → 32x32x32
        self.fc1 = nn.Linear(32 * 16 * 16, 128)         # Flatten and FC
        self.fc2 = nn.Linear(128, num_classes)          # Output layer
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Classes: ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']


In [9]:
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100*correct/total:.2f}%")


Epoch 1/10, Loss: 1.2414, Accuracy: 46.88%
Epoch 2/10, Loss: 0.9605, Accuracy: 63.06%
Epoch 3/10, Loss: 0.8038, Accuracy: 70.66%
Epoch 4/10, Loss: 0.6777, Accuracy: 74.78%
Epoch 5/10, Loss: 0.5047, Accuracy: 82.59%
Epoch 6/10, Loss: 0.3505, Accuracy: 88.47%
Epoch 7/10, Loss: 0.2461, Accuracy: 92.22%
Epoch 8/10, Loss: 0.1559, Accuracy: 95.62%
Epoch 9/10, Loss: 0.0819, Accuracy: 98.06%
Epoch 10/10, Loss: 0.0545, Accuracy: 98.97%
